In [1]:
import numpy as np
import pandas as pd
import scipy.io
import psycopg2
import os

In [2]:
#Get current working directory
cwd = os.getcwd()
#Build overall path
path1 = os.path.join(cwd,r'mill.mat')
#Load .mat file. Output = dictionary
mill = scipy.io.loadmat(path1, squeeze_me=True)['mill'] # select only use-data, exclude matlab meta data
mill_exp1 = mill[0] #select desired experiment run
#case = 1
#run = 1
#flank wear = 0
#time = 2
#depth of cut = 1.5
#feed = 0.5
#material = 1
array_mill_exp1 = np.concatenate((mill_exp1[7],mill_exp1[8],mill_exp1[9],mill_exp1[10],mill_exp1[11],mill_exp1[12]))
array_mill_exp1 = array_mill_exp1.reshape((9000,6),order='F')
labels = ['smcAC','smcDC','vib_table','vib_spindle','AE_table','AE_spindle']
df_mill_exp1 = pd.DataFrame(data=array_mill_exp1, columns=labels)
print(df_mill_exp1)

         smcAC     smcDC  vib_table  vib_spindle  AE_table  AE_spindle
0    -0.017090  0.625000   0.078125     0.314941  0.087280    0.103760
1     0.263672  0.810547   0.085449     0.301514  0.098267    0.123291
2     0.207520  0.781250   0.078125     0.303955  0.092163    0.104980
3     0.302734  0.849609   0.073242     0.300293  0.095215    0.111084
4     0.239258  1.098633   0.083008     0.299072  0.083008    0.092163
...        ...       ...        ...          ...       ...         ...
8995 -0.656738  1.630859   0.080566     0.308838  0.080566    0.087891
8996 -0.869141  1.625977   0.083008     0.314941  0.085449    0.097656
8997 -0.808105  1.381836   0.275879     0.312500  0.101318    0.106812
8998 -0.732422  1.303711   0.522461     0.317383  0.127563    0.112305
8999 -0.576172  1.528320   0.378418     0.311279  0.123291    0.134277

[9000 rows x 6 columns]


In [3]:
    import psycopg2
    from io import StringIO

    # Connection parameters
    param_dic = {
        "host"      : "localhost",
        "database"  : "sensordata",
        "user"      : "postgres",
        "password"  : "JNt8hSW4l+MJ]p;.g[WG"
    }

    #Credit to Naysan Saran. https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/

    def connect(params_dic):
        """ Connect to the PostgreSQL database server """
        conn = None
        try:
            # connect to the PostgreSQL server
            print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(**params_dic)
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            sys.exit(1) 
        print("Connection successful")
        return conn


    def copy_from_stringio(conn, df, table):
        """
        Here we are going save the dataframe in memory 
        and use copy_from() to copy it to the table
        """
        # save dataframe to an in memory buffer
        buffer = StringIO()
        df.to_csv(buffer, index_label='id', header=False)
        buffer.seek(0)

        cursor = conn.cursor()
        try:
            cursor.copy_from(buffer, table, sep=",")
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:

            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        print("copy_from_stringio() done")
        cursor.close()

In [4]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(df_mill_exp1.keys()) #use column labels for SQL query
string1 = "CREATE TABLE MillingNASA (id int"
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");"
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, df_mill_exp1, "MillingNASA") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful
CREATE TABLE MillingNASA (id int, smcAC decimal, smcDC decimal, vib_table decimal, vib_spindle decimal, AE_table decimal, AE_spindle decimal);
copy_from_stringio() done
